In [22]:
# Incluindo bibliotecas
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import random

In [23]:
# Gerando a rede biológica
archive = './HPRD.txt'
Graph = nx.read_edgelist(archive, delimiter = ' ', data = ('pacients',[]))
for i in Graph.nodes:
    Graph.nodes[i]['pacients'] = []

#Abrindo o arquivo de Genes mutados por pacientes
with open('./ProjetoIC/Output_Files/out.txt') as f:
    lines = f.readlines()
    
# Criando uma tabela de Genes por pacientes
TabelaPacienteGene = {'patient': [], 'mutations': []}
for line in lines:
    gene = line.split()[0]
    lista = line.split()[1:]
    for j in lista:
        if j in TabelaPacienteGene['patient']:
            TabelaPacienteGene['mutations'][TabelaPacienteGene['patient'].index(j)] += " " + gene
        else:
            TabelaPacienteGene['patient'].append(j)
            TabelaPacienteGene['mutations'].append(gene)

In [ ]:
tamTeste = int(len(TabelaPacienteGene['pacient'])*0.1)
tamTreinamento = int(len(TabelaPacienteGene['pacient'])*0.72)
tamValidacao = int(len(TabelaPacienteGene['pacient']) - tamTreinamento - tamTeste)
Teste = {'pacient': [], 'mutations': []}
Treinamento = {'pacient': [], 'mutations': []}
Validacao = {'pacient': [], 'mutations': []}
Teste['pacient'] = TabelaPacienteGene['pacient'][:tamTeste]
Teste['mutations'] = TabelaPacienteGene['mutations'][:tamTeste]
Treinamento['pacient'] = TabelaPacienteGene['pacient'][tamTeste:tamTreinamento]
Treinamento['mutations'] = TabelaPacienteGene['mutations'][tamTeste:tamTreinamento]
Validacao['pacient'] = TabelaPacienteGene['pacient'][tamTreinamento:]
Validacao['mutations'] = TabelaPacienteGene['mutations'][tamTreinamento:]

In [43]:
# Versão 1.7 (1.6 generalizando como funcao)
def AlgoritmoV1_6(alpha, GraphModify):
    ini = time.time()
    # Gera subgrafo com o melhor de todos (aleatorizar para os 5 melhores)
    
    ordem = [0,0,0,0,0]
    bestNodes = ["","","","",""]
    AuxiliarGraph = Graph.copy()

    for i in range(0,5,1):
        bestNode = ""
        for node in AuxiliarGraph:
            num = len(AuxiliarGraph.nodes[node]['pacients'])
            if (num > ordem[i]):
                bestNodes[i] = node
                ordem[i] = num
        AuxiliarGraph.remove_node(bestNodes[i])
    
    randNumber = random.randint(0,np.sum(ordem))
    escolhido = 0
    soma = 0
    soma += ordem[escolhido]
    while(randNumber > soma):
        escolhido += 1
        soma += ordem[escolhido]

    Gh = nx.Graph()
    Gh.add_node(bestNodes[escolhido])
    print("O Escolhido foi o " + bestNodes[escolhido])
    # Descobre quantos pacients existem no total no grafo
    listPatientsInGraph = []
    for node in Graph:
        listPatientsInGraph += GraphModify.nodes[node]['pacients']
    listPatientsInGraph = list(dict.fromkeys(listPatientsInGraph))
    maxPatients = len(listPatientsInGraph)

    antigaMelhorNota = np.inf

    listaParaNaoVerificar = []
    TotalVerification = 0
    
    while(True):
        # Criar lista com os pacientes cobertos por Gh na interação atual
        listPatientsInGh = []
        for node in Gh:
            listPatientsInGh += GraphModify.nodes[node]['pacients']
        listPatientsInGh = list(dict.fromkeys(listPatientsInGh))
        patientsInGh = len(listPatientsInGh)
        # Define o tamanho de Gh na interação atual
        sizeOfGh = len(Gh.nodes)

        menorNota = np.inf

        nodesNewSubgraph = list(Gh.nodes)

        listOfSon = []
        for node in Gh.nodes:
            neighborsOfNode = GraphModify.neighbors(node)
            listOfSon += [son for son in neighborsOfNode if son not in Gh if son not in listaParaNaoVerificar]

        listOfSon = list(dict.fromkeys(listOfSon))

        for son in listOfSon:

            possoEliminar = 1

            listPatientsInSon = GraphModify.nodes[son]['pacients']

            neighborsOfSon = list(GraphModify.neighbors(son))

            listOfGrandSon = []

            listOfGrandSon += [grandSon for grandSon in neighborsOfSon if grandSon not in Gh  if grandSon not in listaParaNaoVerificar]
            listOfGrandSon = list(dict.fromkeys(listOfGrandSon))

            difference_son = len([item for item in listPatientsInSon if item not in listPatientsInGh])

            if(difference_son != 0):
                possoEliminar = 0

                nota = alpha*(maxPatients - (patientsInGh + difference_son)) + (1-alpha)*(sizeOfGh + 1)
                TotalVerification += 1
                if(nota < menorNota):
                    menorNota = nota
                    nodesNewSubgraph = list(Gh.nodes) + [son]

            for grandSon in listOfGrandSon:

                listPatientsInGrandSon = GraphModify.nodes[grandSon]['pacients']                                                          

                difference_grandSon = len([item for item in listPatientsInGrandSon if item not in listPatientsInGh if item not in listPatientsInSon])

                if((difference_grandSon + difference_son) != 0):
                    possoEliminar = 0 
                    
                    nota = alpha*(maxPatients - (patientsInGh + difference_son + difference_grandSon)) + (1-alpha)*(sizeOfGh + 2)
                    TotalVerification += 1
                    if(nota < menorNota):
                        menorNota = nota
                        nodesNewSubgraph = list(Gh.nodes) + [son] + [grandSon]

            if(possoEliminar == 1):
                listaParaNaoVerificar += [son]

        Gh = nx.subgraph(GraphModify,nodesNewSubgraph)
        if(menorNota == np.inf or menorNota >= antigaMelhorNota):
            menorNota = antigaMelhorNota;
            break;
        antigaMelhorNota = menorNota;
    return Gh

In [45]:
counterGenes = {'gene':[],'frequency':[]}
numberInterations = 1000
tempTotal = time.time()
for i in range(0,numberInterations,1):
    init = time.time()
    GraphModify = Graph.copy()
    # Alimentando o Grafo com dados do MAF
    tamTotal = len(TabelaPacienteGene['patient'])
    TabelaPacienteGeneTemporaria = TabelaPacienteGene
    for i in range(0,int(tamTotal*0.85),1):
        index = random.randint(0,tamTotal-i-1)
        for gene in TabelaPacienteGeneTemporaria['mutations'][index].split():
            if gene in GraphModify:
                GraphModify.nodes[gene]['pacients'] += [TabelaPacienteGeneTemporaria['patient'][index]]
        TabelaPacienteGeneTemporaria['patient'].pop(index)
        TabelaPacienteGeneTemporaria['mutations'].pop(index)
    tempoAlgoritmo = time.time()
    GraphTest = AlgoritmoV1_6(0.6, GraphModify)
    print("Tempo do algoritmo foi de: " + str(time.time() - tempoAlgoritmo))
    nodesQueApareceram = GraphTest.nodes
    for j in nodesQueApareceram:
        achou = 0
        for k in counterGenes['gene']:
            if( k == j ):
                index = counterGenes['gene'].index(j)
                counterGenes['frequency'][index] += 1
                achou = 1
        if(achou == 0):
            counterGenes['gene'].append(j)
            counterGenes['frequency'].append(1)
    print("Tempo na interacao " + str(i) + " foi de: " + str(time.time() - init))
for i in range(0,len(counterGenes['frequency']),1):
    counterGenes['frequency'][i] /= numberInterations
print("Tempo Final foi de: " + str(time.time() - tempTotal))
counterGenes

O Escolhido foi o MUC16
Tempo do algoritmo foi de: 0.1227729320526123
Tempo na interacao 0 foi de: 0.20754051208496094
O Escolhido foi o TP53
Tempo do algoritmo foi de: 25.152104377746582
Tempo na interacao 1 foi de: 25.359131574630737
O Escolhido foi o BRAF
Tempo do algoritmo foi de: 24.708584547042847
Tempo na interacao 2 foi de: 24.81638765335083
O Escolhido foi o TTN
Tempo do algoritmo foi de: 20.379873037338257
Tempo na interacao 3 foi de: 20.470420360565186
O Escolhido foi o RYR2
Tempo do algoritmo foi de: 21.732121229171753
Tempo na interacao 4 foi de: 21.950297594070435
O Escolhido foi o BRAF
Tempo do algoritmo foi de: 24.22115182876587
Tempo na interacao 5 foi de: 24.3514621257782
O Escolhido foi o TTN
Tempo do algoritmo foi de: 22.69222593307495
Tempo na interacao 6 foi de: 22.819213151931763
O Escolhido foi o TP53
Tempo do algoritmo foi de: 24.654743671417236
Tempo na interacao 7 foi de: 24.746176719665527
O Escolhido foi o TTN
Tempo do algoritmo foi de: 21.743732690811157
T

O Escolhido foi o TTN
Tempo do algoritmo foi de: 16.499221086502075
Tempo na interacao 71 foi de: 16.57229733467102
O Escolhido foi o TP53
Tempo do algoritmo foi de: 18.934895992279053
Tempo na interacao 72 foi de: 19.008064031600952
O Escolhido foi o MUC16
Tempo do algoritmo foi de: 0.11572504043579102
Tempo na interacao 73 foi de: 0.18959403038024902
O Escolhido foi o TTN
Tempo do algoritmo foi de: 16.867037773132324
Tempo na interacao 74 foi de: 16.942779302597046
O Escolhido foi o TTN
Tempo do algoritmo foi de: 16.531465530395508
Tempo na interacao 75 foi de: 16.604773998260498
O Escolhido foi o MUC16
Tempo do algoritmo foi de: 0.11840653419494629
Tempo na interacao 76 foi de: 0.18938684463500977
O Escolhido foi o RYR2
Tempo do algoritmo foi de: 17.22374987602234
Tempo na interacao 77 foi de: 17.3031063079834
O Escolhido foi o TTN
Tempo do algoritmo foi de: 16.59699583053589
Tempo na interacao 78 foi de: 16.671185731887817
O Escolhido foi o BRAF
Tempo do algoritmo foi de: 18.978534

O Escolhido foi o BRAF
Tempo do algoritmo foi de: 18.792420148849487
Tempo na interacao 141 foi de: 18.86476492881775
O Escolhido foi o TTN
Tempo do algoritmo foi de: 16.284910202026367
Tempo na interacao 142 foi de: 16.3661367893219
O Escolhido foi o TP53
Tempo do algoritmo foi de: 18.59424638748169
Tempo na interacao 143 foi de: 18.667272806167603
O Escolhido foi o TP53
Tempo do algoritmo foi de: 18.82275700569153
Tempo na interacao 144 foi de: 18.99811553955078
O Escolhido foi o TTN
Tempo do algoritmo foi de: 16.324431657791138
Tempo na interacao 145 foi de: 16.398721933364868
O Escolhido foi o RYR2
Tempo do algoritmo foi de: 17.21304988861084
Tempo na interacao 146 foi de: 17.285731315612793
O Escolhido foi o TP53
Tempo do algoritmo foi de: 18.67585515975952
Tempo na interacao 147 foi de: 18.749340057373047
O Escolhido foi o TTN
Tempo do algoritmo foi de: 16.988436222076416
Tempo na interacao 148 foi de: 17.059050798416138
O Escolhido foi o TTN
Tempo do algoritmo foi de: 16.5351157

Tempo do algoritmo foi de: 19.274358987808228
Tempo na interacao 210 foi de: 19.355060815811157
O Escolhido foi o RYR2
Tempo do algoritmo foi de: 16.84443759918213
Tempo na interacao 211 foi de: 16.922003030776978
O Escolhido foi o MUC16
Tempo do algoritmo foi de: 0.11845278739929199
Tempo na interacao 212 foi de: 0.18867707252502441
O Escolhido foi o TP53
Tempo do algoritmo foi de: 18.977001667022705
Tempo na interacao 213 foi de: 19.158650159835815
O Escolhido foi o BRAF
Tempo do algoritmo foi de: 18.985527276992798
Tempo na interacao 214 foi de: 19.058058738708496
O Escolhido foi o TP53
Tempo do algoritmo foi de: 18.843531608581543
Tempo na interacao 215 foi de: 18.91711187362671
O Escolhido foi o BRAF
Tempo do algoritmo foi de: 18.55838632583618
Tempo na interacao 216 foi de: 18.737762451171875
O Escolhido foi o RYR2
Tempo do algoritmo foi de: 16.672361850738525
Tempo na interacao 217 foi de: 16.74457097053528
O Escolhido foi o TP53
Tempo do algoritmo foi de: 18.951581716537476
Tem

Tempo do algoritmo foi de: 18.981430768966675
Tempo na interacao 280 foi de: 19.05176305770874
O Escolhido foi o TTN
Tempo do algoritmo foi de: 16.27578067779541
Tempo na interacao 281 foi de: 16.347075700759888
O Escolhido foi o TP53
Tempo do algoritmo foi de: 18.530269622802734
Tempo na interacao 282 foi de: 18.601242780685425
O Escolhido foi o RYR2
Tempo do algoritmo foi de: 16.554261445999146
Tempo na interacao 283 foi de: 16.629387140274048
O Escolhido foi o TP53
Tempo do algoritmo foi de: 19.55307650566101
Tempo na interacao 284 foi de: 19.625338554382324
O Escolhido foi o TP53
Tempo do algoritmo foi de: 18.88837504386902
Tempo na interacao 285 foi de: 18.963256359100342
O Escolhido foi o RYR2
Tempo do algoritmo foi de: 16.699509143829346
Tempo na interacao 286 foi de: 16.769758462905884
O Escolhido foi o BRAF
Tempo do algoritmo foi de: 19.65760111808777
Tempo na interacao 287 foi de: 19.730870962142944
O Escolhido foi o TP53
Tempo do algoritmo foi de: 18.7172634601593
Tempo na i

...

Tempo do algoritmo foi de: 16.731281518936157
Tempo na interacao 768 foi de: 16.806909322738647
O Escolhido foi o TP53
Tempo do algoritmo foi de: 18.652782917022705
Tempo na interacao 769 foi de: 18.729072093963623
O Escolhido foi o TTN
Tempo do algoritmo foi de: 16.378841161727905
Tempo na interacao 770 foi de: 16.450913429260254
O Escolhido foi o TTN
Tempo do algoritmo foi de: 16.218883514404297
Tempo na interacao 771 foi de: 16.396796703338623
O Escolhido foi o TP53
Tempo do algoritmo foi de: 18.628356218338013
Tempo na interacao 772 foi de: 18.701075792312622
O Escolhido foi o BRAF
Tempo do algoritmo foi de: 18.740442037582397
Tempo na interacao 773 foi de: 18.810486793518066
O Escolhido foi o TP53
Tempo do algoritmo foi de: 18.858100652694702
Tempo na interacao 774 foi de: 18.93171787261963
O Escolhido foi o TP53
Tempo do algoritmo foi de: 19.023173809051514
Tempo na interacao 775 foi de: 19.098718881607056
O Escolhido foi o MUC16
Tempo do algoritmo foi de: 0.23487567901611328
Tem

Tempo do algoritmo foi de: 22.110021829605103
Tempo na interacao 838 foi de: 22.20159125328064
O Escolhido foi o TTN
Tempo do algoritmo foi de: 21.67367458343506
Tempo na interacao 839 foi de: 21.80823040008545
O Escolhido foi o TP53
Tempo do algoritmo foi de: 23.710662364959717
Tempo na interacao 840 foi de: 23.806929349899292
O Escolhido foi o MUC16
Tempo do algoritmo foi de: 0.1551976203918457
Tempo na interacao 841 foi de: 0.26006460189819336
O Escolhido foi o TTN
Tempo do algoritmo foi de: 21.15302586555481
Tempo na interacao 842 foi de: 21.24388885498047
O Escolhido foi o TP53
Tempo do algoritmo foi de: 23.616501569747925
Tempo na interacao 843 foi de: 23.707005739212036
O Escolhido foi o RYR2
Tempo do algoritmo foi de: 22.202598094940186
Tempo na interacao 844 foi de: 22.29619812965393
O Escolhido foi o TP53
Tempo do algoritmo foi de: 27.032732725143433
Tempo na interacao 845 foi de: 27.153444528579712
O Escolhido foi o TP53
Tempo do algoritmo foi de: 45.7739999294281
Tempo na i

O Escolhido foi o TTN
Tempo do algoritmo foi de: 16.394985675811768
Tempo na interacao 908 foi de: 16.46712374687195
O Escolhido foi o MUC16
Tempo do algoritmo foi de: 0.12277984619140625
Tempo na interacao 909 foi de: 0.19768047332763672
O Escolhido foi o TTN
Tempo do algoritmo foi de: 16.484510898590088
Tempo na interacao 910 foi de: 16.560850858688354
O Escolhido foi o MUC16
Tempo do algoritmo foi de: 0.2301654815673828
Tempo na interacao 911 foi de: 0.30356383323669434
O Escolhido foi o RYR2
Tempo do algoritmo foi de: 17.097137212753296
Tempo na interacao 912 foi de: 17.166054248809814
O Escolhido foi o TP53
Tempo do algoritmo foi de: 18.625754833221436
Tempo na interacao 913 foi de: 18.696579217910767
O Escolhido foi o RYR2
Tempo do algoritmo foi de: 16.933518886566162
Tempo na interacao 914 foi de: 17.112481832504272
O Escolhido foi o MUC16
Tempo do algoritmo foi de: 0.11366820335388184
Tempo na interacao 915 foi de: 0.18521928787231445
O Escolhido foi o TTN
Tempo do algoritmo fo

Tempo do algoritmo foi de: 18.845752477645874
Tempo na interacao 977 foi de: 18.916263341903687
O Escolhido foi o TP53
Tempo do algoritmo foi de: 18.786925554275513
Tempo na interacao 978 foi de: 18.86282992362976
O Escolhido foi o TTN
Tempo do algoritmo foi de: 16.298554182052612
Tempo na interacao 979 foi de: 16.371955633163452
O Escolhido foi o TTN
Tempo do algoritmo foi de: 16.315542697906494
Tempo na interacao 980 foi de: 16.50412607192993
O Escolhido foi o BRAF
Tempo do algoritmo foi de: 18.948811531066895
Tempo na interacao 981 foi de: 19.02277374267578
O Escolhido foi o RYR2
Tempo do algoritmo foi de: 17.116740465164185
Tempo na interacao 982 foi de: 17.19002866744995
O Escolhido foi o BRAF
Tempo do algoritmo foi de: 18.767956972122192
Tempo na interacao 983 foi de: 18.952980995178223
O Escolhido foi o MUC16
Tempo do algoritmo foi de: 0.11040735244750977
Tempo na interacao 984 foi de: 0.1786947250366211
O Escolhido foi o BRAF
Tempo do algoritmo foi de: 19.400394439697266
Tempo 

{'gene': ['MSLN',
  'MUC16',
  'LGALS1',
  'VPREB1',
  'PIK3CG',
  'POU2AF1',
  'ARHGAP4',
  'CAV1',
  'SPR',
  'CSF2RB',
  'HLF',
  'JAK2',
  'PIK3R5',
  'PLD2',
  'RPTOR',
  'CAMK2G',
  'ATN1',
  'NPAS2',
  'CREBBP',
  'NFE2L2',
  'MAPK3',
  'SMARCA5',
  'HRAS',
  'COPS6',
  'ZCCHC10',
  'GIGYF1',
  'ADAM15',
  'CHD3',
  'TBP',
  'HCK',
  'DDC',
  'PPP1R13L',
  'KRAS',
  'POU2F2',
  'GRIN2B',
  'GRM8',
  'ETS1',
  'SDCCAG3',
  'RYR1',
  'IRS2',
  'KIT',
  'SNX9',
  'ELMO1',
  'SMARCA4',
  'RASSF2',
  'HSP90AA1',
  'PTPN5',
  'RYR2',
  'PTPRR',
  'BRAF',
  'NCK1',
  'INPP5D',
  'DUSP5',
  'PIK3R1',
  'TNS1',
  'ARRB1',
  'PPID',
  'JAK1',
  'TDRD7',
  'LEPR',
  'HIST4H4',
  'ARHGEF6',
  'DCC',
  'RAF1',
  'GRIN1',
  'RASA1',
  'TP53BP1',
  'KIF15',
  'PTK2',
  'WDR62',
  'PRKG1',
  'LAMA4',
  'SH3KBP1',
  'NRAS',
  'ZFPM2',
  'TEAD4',
  'ATR',
  'CDKN2A',
  'PIK3CA',
  'TP73',
  'ATM',
  'SKAP2',
  'UBE2A',
  'POU2F3',
  'NGFR',
  'MAPK1',
  'PRRC2B',
  'SPTA1',
  'PDE5A',
  'BIRC6',


In [49]:
counterGenes['frequency'][0]

0.173